<a href="https://colab.research.google.com/github/PedroM2626/Pet-Classifier-Dime-Frida/blob/main/dime_frida_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Configuração Inicial e Importações
Configura o logger para depuração e importa as bibliotecas necessárias.

In [12]:
import logging
import os
import sys
import random
import numpy as np
import tensorflow as tf
import matplotlib
matplotlib.use('Agg') # Usar o backend 'Agg' para evitar problemas de exibição e salvamento
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import cv2
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
import keras
from matplotlib.pyplot import imshow
from tensorflow.keras.layers import Dense, Flatten

# 1. Detecta se o ambiente é o Google Colab
if 'google.colab' in sys.modules:
    print("Ambiente Colab detectado. Iniciando download da pasta 'dataset'...")

    # Nome do repositório e pasta
    repo_url = "https://github.com/PedroM2626/Pet-Classifier-Dime-Frida.git"
    folder_name = "my_precious-dataset"

    # Comando para clonar apenas a pasta específica usando sparse-checkout
    !git clone --depth 1 --filter=blob:none --sparse {repo_url}
    %cd Pet-Classifier-Dime-Frida
    !git sparse-checkout set {folder_name}

    # Move a pasta para o diretório principal do Colab para facilitar o acesso
    !mv {folder_name} /content/
    %cd /content/

    # Limpeza: remove o resto do repositório clonado
    !rm -rf Pet-Classifier-Dime-Frida

    print("\n✅ Pasta 'dataset' baixada com sucesso e disponível em /content/dataset")
else:
    print("Ambiente local detectado. Certifique-se de ter os dados manualmente ou via git.")

# Configurar o logger
logging.basicConfig(filename='debug_log.txt', level=logging.DEBUG,
                    format='%(asctime)s - %(levelname)s - %(message)s')

logging.info("Script dime_frida_classifier.py iniciado.")

Ambiente Colab detectado. Iniciando download da pasta 'dataset'...
Cloning into 'Pet-Classifier-Dime-Frida'...
remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 7 (delta 0), reused 5 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (7/7), done.
remote: Enumerating objects: 5, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 5 (delta 0), reused 4 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (5/5), 159.40 KiB | 4.19 MiB/s, done.
/content/Pet-Classifier-Dime-Frida
remote: Enumerating objects: 51, done.
remote: Counting objects: 100% (51/51), done.
remote: Compressing objects: 100% (51/51), done.
remote: Total 51 (delta 0), reused 51 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (51/51), 6.66 MiB | 29.40 MiB/s, done.
mv: cannot move 'my_precious-dataset' to '/content/my_precious-dataset': Directory not e

# Funções Auxiliares
Define funções para carregar imagens e gerar mapas de calor Grad-CAM.

In [13]:
# helper function to load image and return it and input vector
def get_image(path):
    try:
        img = image.load_img(path, target_size=(224, 224))
        x = image.img_to_array(img)
        x = preprocess_input(x)
        return img, x
    except Exception as e:
        logging.warning(f"Aviso: Não foi possível carregar a imagem {path}. Erro: {e}")
        return None, None

def make_gradcam_heatmap(img_array, model, last_conv_layer_name, pred_index=None):
    # First, we create a model that maps the input image to the activations
    # of the last convolutional layer as well as the output predictions
    grad_model = tf.keras.models.Model(
        [model.inputs], [model.get_layer(last_conv_layer_name).output, model.output]
    )

    # Then, we compute the gradient of the top predicted class for our input image
    # with respect to the activations of the last convolutional layer
    with tf.GradientTape() as tape:
        last_conv_layer_output, preds = grad_model(img_array)
        if pred_index is None:
            pred_index = tf.argmax(preds[0])
        class_channel = preds[:, pred_index]

    # This is the gradient of the output neuron (top predicted or chosen) with respect
    # to the output feature map of the last conv layer
    grads = tape.gradient(class_channel, last_conv_layer_output)

    # This is a vector where each entry is the mean intensity of the gradient
    # over a specific feature map channel
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

    # We multiply each channel in the feature map array by "how important this channel is"
    # with respect to the top predicted class
    # then sum all the channels to obtain the heatmap class activation
    last_conv_layer_output = last_conv_layer_output[0]
    heatmap = last_conv_layer_output @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)

    # For visualization purpose, we will normalize the heatmap between 0 & 1
    heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)
    return heatmap.numpy()

# Configurações do Dataset
Define o caminho raiz do dataset e as categorias de imagens.

In [14]:
# --- Configurações do Dataset ---
if 'google.colab' in sys.modules:
  root = '/content/my_precious-dataset'
else:
  root = r'.\my_precious-dataset'

logging.info(f"Caminho root configurado: {root}")
train_split = 0.8 # 80% para treinamento
val_split = 0.2   # 20% para validação

# Definir as categorias explicitamente
categories = [os.path.join(root, 'Dime'), os.path.join(root, 'Frida')]
category_names = ['Dime', 'Frida'] # Nomes das classes para exibição

logging.info(f"Categorias encontradas: {category_names}")

# Carregamento e Pré-processamento dos Dados
Carrega as imagens, as pré-processa e divide em conjuntos de treinamento, validação e teste.

In [15]:
# --- Carregamento e Pré-processamento dos Dados ---
data = []
logging.info(f"Iniciando carregamento de imagens para as categorias: {category_names}")
for c_idx, category_path in enumerate(categories):
    images = [os.path.join(dp, f) for dp, dn, filenames
              in os.walk(category_path) for f in filenames
              if os.path.splitext(f)[1].lower() in ['.jpg','.png','.jpeg']]
    logging.info(f"Número de imagens encontradas em {category_path}: {len(images)}")
    for img_path in images:
        img, x = get_image(img_path)
        if img is not None and x is not None:
              data.append({'x':x, 'y':c_idx})

logging.info(f"Comprimento final da lista de dados após o carregamento: {len(data)}")

# Contar o número de classes
num_classes = len(category_names)

# Randomizar a ordem dos dados
random.shuffle(data)

# Dividir os dados em conjuntos de treinamento, validação e teste (70%, 15%, 15%)
random.shuffle(data)
train_split_ratio = 0.7
val_split_ratio = 0.15

idx_train = int(len(data) * train_split_ratio)
idx_val = int(len(data) * (train_split_ratio + val_split_ratio))

train = data[:idx_train]
val = data[idx_train:idx_val]
test = data[idx_val:]

# Separar dados para rótulos
x_train_list = []
y_train_list = []
for t in train:
    x_train_list.append(t["x"])
    y_train_list.append(t["y"])
x_train = np.array(x_train_list)
y_train = np.array(y_train_list)

x_val_list = []
y_val_list = []
for t in val:
    x_val_list.append(t["x"])
    y_val_list.append(t["y"])
x_val = np.array(x_val_list)
y_val = np.array(y_val_list)

x_test_list = []
y_test_list = []
for t in test:
    x_test_list.append(t["x"])
    y_test_list.append(t["y"])
x_test = np.array(x_test_list)
y_test = np.array(y_test_list)

# Normalizar dados
x_train = x_train.astype('float32') / 255.
x_val = x_val.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.

# Converter rótulos para vetores one-hot
y_train = keras.utils.to_categorical(y_train, num_classes)
y_val = keras.utils.to_categorical(y_val, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

# Resumo
print("Carregadas %d imagens de %d categorias"%(len(data), num_classes))
logging.info(f"Número de amostras de treinamento: {len(train)}")
logging.info(f"Número de amostras de validação: {len(val)}")
logging.info(f"Número de amostras de teste: {len(test)}")

Carregadas 44 imagens de 2 categorias


# Construção e Treinamento do Modelo (VGG16 com Transfer Learning)
Configura e treina um modelo VGG16 para classificação de imagens usando transfer learning.

In [16]:
# --- Construção e Treinamento do Modelo (VGG16 com Transfer Learning) ---
print("\n--- Usando VGG16 para Transfer Learning ---")
vgg = keras.applications.VGG16(weights='imagenet', include_top=True)

inp = vgg.input
new_classification_layer = Dense(num_classes, activation='softmax')
out = new_classification_layer(vgg.layers[-2].output)
model_new = Model(inp, out)

# Congelar todas as camadas, exceto a última
for l, layer in enumerate(model_new.layers[:-1]):
    layer.trainable = False
for l, layer in enumerate(model_new.layers[-1:]):
    layer.trainable = True

model_new.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model_new.summary()

logging.info(f"Shape de x_train: {x_train.shape}")
logging.info(f"Shape de y_train: {y_train.shape}")
logging.info(f"Shape de x_val: {x_val.shape}")
logging.info(f"Shape de y_val: {y_val.shape}")

history2 = model_new.fit(x_train, y_train,
                         batch_size=128,
                         epochs=10,
                         validation_data=(x_val, y_val))


--- Usando VGG16 para Transfer Learning ---


Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv1 (Conv2D)           │ (None, 224, 224, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv2D)           │ (None, 224, 224, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling2D)      │ (None, 112, 112, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv2D)           │ (None, 112, 112, 128)  │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv2D)           │ (None, 112, 112, 128)  │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling2D)      │ (None, 56, 56, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv2D)           │ (None, 56, 56, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv3 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_pool (MaxPooling2D)      │ (None, 28, 28, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv1 (Conv2D)           │ (None, 28, 28, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv2 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv3 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_pool (MaxPooling2D)      │ (None, 14, 14, 512)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv1 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv2 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv3 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_pool (MaxPooling2D)      │ (None, 7, 7, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ fc1 (Dense)                     │ (None, 4096)           │   102,764,544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ fc2 (Dense)                     │ (None, 4096)           │    16,781,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 2)              │         8,194 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 134,268,738 (512.19 MB)

 Trainable params: 8,194 (32.01 KB)

 Non-trainable params: 134,260,544 (512.16 MB)

Epoch 1/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 26s 26s/step - accuracy: 0.5667 - loss: 0.8381 - val_accuracy: 0.7143 - val_loss: 0.8277
Epoch 2/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 28s 28s/step - accuracy: 0.4333 - loss: 1.4587 - val_accuracy: 0.7143 - val_loss: 0.6272
Epoch 3/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 23s 23s/step - accuracy: 0.4333 - loss: 0.8321 - val_accuracy: 0.2857 - val_loss: 1.1427
Epoch 4/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 43s 43s/step - accuracy: 0.5667 - loss: 0.7586 - val_accuracy: 0.2857 - val_loss: 1.6170
Epoch 5/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 40s 40s/step - accuracy: 0.5667 - loss: 0.9827 - val_accuracy: 0.2857 - val_loss: 1.4384
Epoch 6/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 40s 40s/step - accuracy: 0.5667 - loss: 0.8805 - val_accuracy: 0.2857 - val_loss: 0.9274
Epoch 7/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 43s 43s/step - accuracy: 0.5667 - loss: 0.6300 - val_accuracy: 0.8571 - val_loss: 0.5739
Epoch 8/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 26s 26s/step - accuracy: 0.8333 - loss: 0.5673 - val_accuracy: 0.7143 - val_loss: 0.5287


# Avaliação do Modelo
Avalia o desempenho do modelo treinado no conjunto de testes.

In [17]:
# --- Avaliação do Modelo ---
loss, accuracy = model_new.evaluate(x_test, y_test, verbose=0)
print('Perda no teste:', loss)
print('Precisão no teste:', accuracy)

Perda no teste: 0.6861063241958618
Precisão no teste: 0.5714285969734192


# Análise de Imagem da Pasta 'tests'
Processa imagens da pasta 'tests', faz previsões e salva os resultados.

In [18]:
# --- Análise de Imagem da Pasta 'tests' ---
logging.info("--- Análise de Imagens da Pasta 'tests' ---")
test_images_dir = os.path.join(root, 'tests')
output_dir = os.path.join(root, 'results')

# Certificar-se de que o diretório de saída existe
os.makedirs(output_dir, exist_ok=True)

image_files = [f for f in os.listdir(test_images_dir) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]

if not image_files:
    logging.warning(f"Nenhuma imagem encontrada na pasta de testes: {test_images_dir}")
    print(f"Nenhuma imagem encontrada na pasta de testes: {test_images_dir}")
else:
    for image_filename in image_files:
        img_path = os.path.join(test_images_dir, image_filename)
        logging.debug(f"Processando imagem de teste: {img_path}")
        img, img_processed = get_image(img_path)

        if img is not None and img_processed is not None:
            img_processed = np.expand_dims(img_processed, axis=0)
            prediction = model_new.predict(img_processed)
            predicted_class_idx = np.argmax(prediction)
            predicted_class_name = category_names[predicted_class_idx]

            # Salvar as imagens
            fig, ax = plt.subplots(1, 1, figsize=(7, 5))

            ax.imshow(img)
            ax.set_title(f"Predicted: {predicted_class_name}")
            ax.axis("off")

            output_filename = os.path.splitext(image_filename)[0] + ".png"
            full_output_path = os.path.join(output_dir, output_filename)
            try:
                plt.savefig(full_output_path)
            except Exception as e:
                logging.error(f"Erro ao salvar a imagem {image_filename} em {full_output_path}: {e}")
            plt.close(fig) # Fechar a figura para liberar memória
        else:
            print(f"Não foi possível carregar ou processar a imagem de teste: {img_path}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 824ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 608ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 609ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 625ms/step
